In [4]:
import numpy as np
import pandas as pd
import nltk
import json

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline

from luwiji.text_proc import illustration, demo

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from string import punctuation

nltk.download("stopwords")

sw_indo = stopwords.words("indonesian") + list(punctuation) + ["``"]
import warnings

# Menekan peringatan
warnings.filterwarnings("ignore", category=UserWarning, module="sklearn")


[nltk_data] Downloading package stopwords to /Users/izzal/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### Import Data

In [5]:
with open('../assets/intents.json') as file:
    data = json.load(file)

# Ekstrak kolom "tag" dan "patterns" dari JSON bersarang
selected_data = []

for intent in data["intents"]:
    tag = intent["tag"]
    patterns = intent["patterns"]
    
    for pattern in patterns:
        selected_data.append({"tag": tag, "patterns": pattern})

# Buat DataFrame dari data yang diambil
df = pd.DataFrame(selected_data)
df.head()

,tag,patterns
0,notfound,
1,notfound,
2,sapaan,Halo
3,sapaan,Hai
4,sapaan,Selamat pagi


### Dataset Splitting

In [6]:
X = df.patterns
y = df.tag

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.23, stratify=y, random_state=42)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((268,), (81,), (268,), (81,))

In [7]:
df.groupby('tag')['patterns'].count().reset_index()

,tag,patterns
0,EyeU,5
1,alergi mata,4
2,alkohol,3
3,anak-anak katarak,4
4,aplikasi,4
...,...,...
76,tidak,10
77,tidur,3
78,usia,3
79,vitamin,4


### Training

In [8]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import GridSearchCV
from jcopml.tuning import random_search_params as rsp


#### KNN Clasifier

In [9]:
pipeline = Pipeline([
    ('prep', TfidfVectorizer(tokenizer=word_tokenize, stop_words=sw_indo, ngram_range=(1,3))),
    ('algo', KNeighborsClassifier()),
])

params = {
    "prep__ngram_range": [(1, 1), (1, 2), (1, 3)],
    "algo__n_neighbors": range(1,11,2),
    "algo__weights":["distance"],
    "algo__metric":["manhattan","euclidean"]
}

model_knn = GridSearchCV(pipeline, params, cv=3, n_jobs=-1, verbose=0)
model_knn.fit(X_train, y_train)

print(model_knn.best_params_)
print(model_knn.score(X_train, y_train), model_knn.best_score_, model_knn.score(X_test,y_test))

/opt/homebrew/Caskroom/miniconda/base/envs/venv/lib/python3.11/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/opt/homebrew/Caskroom/miniconda/base/envs/venv/lib/python3.11/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/opt/homebrew/Caskroom/miniconda/base/envs/venv/lib/python3.11/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/opt/homebrew/Caskroom/miniconda/base/envs/venv/lib/python3.11/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/opt/homebrew/Caskroom/miniconda/base/envs/venv/lib/python3.11/site-packages/sklearn/feature_extract

{'algo__metric': 'euclidean', 'algo__n_neighbors': 3, 'algo__weights': 'distance', 'prep__ngram_range': (1, 1)}
0.9104477611940298 0.6531002913025384 0.654320987654321


In [18]:
model_knn.predict(["apa itu rabun jauh"])

array(['kamu'], dtype=object)

In [14]:
from sklearn.metrics import classification_report
# Melakukan prediksi pada data uji
y_pred = model_knn.predict(X_test)

# Menghasilkan laporan klasifikasi
report = classification_report(y_test, y_pred)

# Menampilkan laporan klasifikasi
print("Classification Report:\n", report)


Classification Report:
                        precision    recall  f1-score   support

                 EyeU       1.00      1.00      1.00         1
          alergi mata       1.00      1.00      1.00         1
              alkohol       0.00      0.00      0.00         1
    anak-anak katarak       0.50      1.00      0.67         1
             aplikasi       1.00      1.00      1.00         1
   aplikasi diagnosis       1.00      1.00      1.00         1
         astigmatisme       1.00      1.00      1.00         1
               cahaya       1.00      1.00      1.00         1
       cegah glaukoma       1.00      1.00      1.00         1
   diagnosis glaukoma       0.00      0.00      0.00         1
    diagnosis katarak       0.00      0.00      0.00         1
              edukasi       1.00      1.00      1.00         1
        faktor resiko       0.00      0.00      0.00         1
  gejala awal katarak       0.33      1.00      0.50         1
      gejala glaukoma       1.

In [16]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test, y_pred)
print(accuracy)

0.654320987654321


In [ ]:
import pickle
# open a file, where you ant to store the data
file = open('../assets/model/knn.pkl', 'wb')

# dump information to that file
pickle.dump(model_knn, file)

# close the file
file.close()